# M2177.003100 Deep Learning <br> Final Proejct: Text to Image Synthesis (Tensorflow)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PPT file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the training process **</font> so that TAs can grade both your code and results.  
**The TA will set a config file as 'eval_birds.yml' when evaluating the code using 'hidden test dataset'. Thus, please make sure that your code can generate proper data to measure inception score and R-precision of 'hidden test dataset'.**

## 1. Load datasets
The Birds dataset will be downloaded automatically if it is not located in the *data* directory. <br>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os, nltk
from miscc.config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz
import numpy as np
import scipy
from utils.data_utils import CUBDataset
from utils.loss import cosine_similarity
import pandas as pd
from scipy.io import loadmat
import re
import string
import random
import time

%matplotlib inline

/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chszerg

In [2]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/train_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = 'sample/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

Using config:
{'BATCH_SIZE': 64,
 'CHECKPOINT_DIR': './checkpoint',
 'CHECKPOINT_NAME': 'model.ckpt',
 'CNN': {'EMBEDDING_DIM': 0, 'H_DIM': 0},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': False,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': False,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 0,
         'DF_DIM': 0,
         'EMBEDDING_DIM': 0,
         'GF_DIM': 0,
         'R_NUM': 0,
         'Z_DIM': 512},
 'GPU_ID': '0',
 'IMAGE_SIZE': 256,
 'NUM_BATCH_FOR_TEST': 0,
 'RANDOM_SEED': 0,
 'RNN': {'EMBEDDING_DIM': 0,
         'H_DIM': 0,
         'TYPE': '',
         'VOCAB_SIZE': 0,
         'WORD_EMBEDDING_DIM': 0},
 'R_PRECISION_DIR': './evaluation',
 'R_PRECISION_FILE': 'r_precision.npz',
 'R_PRECISION_FILE_HIDDEN': 'r_precision_hidden.npz',
 'TEST': {'B_EXAMPLE': False,
          'GENERATED_HIDDEN_TEST_IMAGES': './evaluation/generated_images_hidden',
          'GENERATED_TEST_IMAGES'

/home/chszerg/final-project-deep-learning-19-tf/miscc/config.py:121: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [3]:
train_dataset = CUBDataset(cfg.DATA_DIR, split='train')
test_dataset = CUBDataset(cfg.DATA_DIR, split='test')

self.current_dir:
/home/chszerg/final-project-deep-learning-19-tf

self.data_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011.tgz

Dataset already exists
self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle
self.current_dir:
/home/chszerg/final-project-deep-learning-19-tf

self.data_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011.tgz

Dataset already exists
self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle


In [4]:
train_images = train_dataset.images
test_images = test_dataset.images
train_captions = np.asarray(train_dataset.captions_ids)
test_captions = np.asarray(test_dataset.captions_ids)
print(train_images.shape)
print(test_images.shape)
print(train_captions.shape)
print(test_captions.shape)

(8855, 256, 256, 3)
(2933, 256, 256, 3)
(88550, 20)
(29330, 20)


In [5]:
from skimage.transform import resize
train_images_64 = []
for train_image in train_images:
    train_images_64.append(resize(train_image, (64, 64, 3)))
train_images_64 = np.asarray(train_images_64)
print(train_images_64.shape)
assert train_images_64.shape[0] == train_images.shape[0]
test_images_64 = []
for test_image in test_images:
    test_images_64.append(resize(test_image, (64, 64, 3)))
test_images_64 = np.asarray(test_images_64)
print(test_images_64.shape)
assert test_images_64.shape[0] == test_images.shape[0]

(8855, 64, 64, 3)
(2933, 64, 64, 3)


In [6]:
n_captions_train = len(train_captions)
n_captions_per_image = 10
n_images_train = len(train_images)
print(train_images.shape)

(8855, 256, 256, 3)


In [7]:
import scipy.misc
import threading
import scipy.ndimage as ndi
from skimage import transform
from skimage import exposure
import skimage
from nltk.tokenize import RegexpTokenizer
import scipy.ndimage as ndimage

def sent2ID(sample_sentence):
    caption = []
    cap = sample_sentence
    if len(cap) == 0:
        exit()
    cap = cap.replace("\ufffd\ufffd", " ")
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(cap.lower())
    tokens_new = []
    for t in tokens:
        t = t.encode('ascii', 'ignore').decode('ascii')
        if len(t) > 0:
            tokens_new.append(t)
    caption.append(tokens_new)
    caption_new = []
    t = caption[0]
    rev = []
    for w in t:
        if w in train_dataset.wordtoix:
            rev.append(train_dataset.wordtoix[w])
    x, x_len = train_dataset.get_caption(rev)
    caption_new.append(np.squeeze(x, axis=1))
    return caption_new

def ID2sent(sample_caption):
    sentence = []
    for ID in sample_caption:
        if ID != train_dataset.ixtoword['<PAD>']:
            sentence.append(train_dataset.ixtoword[ID])
    return sentence

def get_random_int(min=0, max=10, number=5):
    return [random.randint(min,max) for p in range(0,number)]

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

def threading_data(data=None, fn=None, **kwargs):
    def apply_fn(results, i, data, kwargs):
        results[i] = fn(data, **kwargs)
    results = [None] * len(data)
    threads = []
    for i in range(len(data)):
        t = threading.Thread(
                        name='threading_and_return',
                        target=apply_fn,
                        args=(results, i, data[i], kwargs)
                        )
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    return np.asarray(results)

def apply_transform(x, transform_matrix, channel_index=2, fill_mode='nearest', cval=0., order=1):
    x = np.rollaxis(x, channel_index, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(x_channel, final_affine_matrix,
                      final_offset, order=order, mode=fill_mode, cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_index + 1)
    return x

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

def rotation(x, rg=20, is_random=False, row_index=0, col_index=1, channel_index=2,
                    fill_mode='nearest', cval=0.):
    if is_random:
        theta = np.pi / 180 * np.random.uniform(-rg, rg)
    else:
        theta = np.pi / 180 * rg
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])
    h, w = x.shape[row_index], x.shape[col_index]
    transform_matrix = transform_matrix_offset_center(rotation_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_index, fill_mode, cval)
    return x

def crop(x, wrg, hrg, is_random=False, row_index=0, col_index=1, channel_index=2):
    h, w = x.shape[row_index], x.shape[col_index]
    assert (h > hrg) and (w > wrg), "The size of cropping should smaller than the original image"
    if is_random:
        h_offset = int(np.random.uniform(0, h-hrg) - 1)
        w_offset = int(np.random.uniform(0, w-wrg) - 1)
        return x[h_offset: hrg + h_offset ,w_offset: wrg + w_offset]
    else:
        h_offset = int(np.floor((h - hrg)/ 2.))
        w_offset = int(np.floor((w - wrg)/ 2.))
        h_end = h_offset + hrg
        w_end = w_offset + wrg
        return x[h_offset: h_end, w_offset: w_end]

def flip_axis(x, axis, is_random=False):
    if is_random:
        factor = np.random.uniform(-1, 1)
        if factor > 0:
            x = np.asarray(x).swapaxes(axis, 0)
            x = x[::-1, ...]
            x = x.swapaxes(0, axis)
            return x
        else:
            return x
    else:
        x = np.asarray(x).swapaxes(axis, 0)
        x = x[::-1, ...]
        x = x.swapaxes(0, axis)
        return x

def imresize(x, size=[100, 100], interp='bilinear', mode=None):
    if x.shape[-1] == 1:
        x = scipy.misc.imresize(x[:, :, 0], size, interp=interp, mode=mode)
        return x[:, :, np.newaxis]
    elif x.shape[-1] == 3:
        return scipy.misc.imresize(x, size, interp=interp, mode=mode)
    else:
        raise Exception("Unsupported channel %d" % x.shape[-1])

def prepro_img(x, mode=None):
    if mode=='train':
        x = flip_axis(x, axis=1, is_random=True)
        x = rotation(x, rg=16, is_random=True, fill_mode='nearest')
        x = imresize(x, size=[256 + 60, 256 + 60], interp='bilinear', mode=None)
        x = crop(x, wrg=256, hrg=256, is_random=True)
        x = x / (255. / 2.)
        x = x - 1.
    return x

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.01
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [8]:
import tensorflow.contrib.slim as slim

def resBlock(x, channels=64, kernel_size=[3,3], scale=1):
    tmp = slim.conv2d(x, channels, kernel_size, activation_fn=None)
    tmp = tf.nn.relu(tmp)
    tmp = slim.conv2d(tmp, channels, kernel_size, activation_fn=None)
    tmp *= scale
    return x + tmp

def upsample(x, scale=2, features=64, activation=tf.nn.relu):
    assert scale in [2,3,4]
    x = slim.conv2d(x, features, [3,3], activation_fn=activation)
    if scale == 2:
        ps_features = 3*(scale**2)
        x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
        x = PS(x, 2, color=True)
    elif scale == 3:
        ps_features =3*(scale**2)
        x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
        x = PS(x, 3, color=True)
    elif scale == 4:
        ps_features = 3*(2**2)
        for i in range(2):
            x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
            x = PS(x, 2, color=True)
    return x

def _phase_shift(I, r):
    bsize, a, b, c = I.get_shape().as_list()
    bsize = tf.shape(I)[0]
    X = tf.reshape(I, (bsize, a, b, r, r))
    X = tf.transpose(X, (0, 1, 2, 4, 3))
    X = tf.split(X, a, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X],2)
    X = tf.split(X, b, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X],2)
    return tf.reshape(X, (bsize, a*r, b*r, 1))

def PS(X, r, color=False):
    if color:
        Xc = tf.split(X, 3, 3)
        X = tf.concat([_phase_shift(x, r) for x in Xc],3)
    else:
        X = _phase_shift(X, r)
    return X

def log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def save(saver, sess, logdir, step):
    model_name = 'model.ckpt'
    checkpoint_path = os.path.join(logdir, model_name)
    if not os.path.exists(logdir):
        os.makedirs(logdir)
    saver.save(sess, checkpoint_path, global_step=step)
    print('The checkpoint has been created.')

def load(saver, sess, ckpt_path):
    saver.restore(sess, ckpt_path)
    print("Restored model parameters from {}".format(ckpt_path))

In [9]:
class EDSR(object):
    def __init__(self, img_size=64, num_layers=32, feature_size=256, scale=4, output_channels=3, lr=1e-4):
        print("Building EDSR...")
        """ Information """
        self.img_size = img_size
        self.scale = scale
        self.output_channels = output_channels
        self.lr = tf.Variable(lr, trainable=False)
        
        """ Place Holders"""
        self.input = x = tf.placeholder(tf.float32, [None, img_size, img_size, output_channels])
        self.target = y = tf.placeholder(tf.float32,[None, img_size*scale, img_size*scale, output_channels])
        image_input = x
        image_target = y
        
        """ Graph """
        x = slim.conv2d(image_input, feature_size, [3,3])
        conv_1 = x
        scaling_factor = 0.1
        for i in range(num_layers):
            x = resBlock(x, feature_size, scale=scaling_factor)
        x = slim.conv2d(x, feature_size, [3,3])
        x += conv_1
        x = upsample(x, scale, feature_size, None)
        self.out = output = tf.nn.tanh(x)
        
        """ Loss and Optimizer """
        self.loss = loss = tf.reduce_mean(tf.losses.absolute_difference(image_target, output))
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.minimize(self.loss)
        print("Building Complete!")

In [11]:
""" Re-Training Parameters """
n_epochs = 100
batch_size = 16
lr = 2e-5
decay_every = 80
lr_decay = 0.5
train_samples_dir = 'EDSR_samples_with_noise'
if os.path.exists(train_samples_dir) == False:
    os.makedirs(train_samples_dir)
checkpoint_dir = './EDSR_checkpoint'

In [12]:
tf.reset_default_graph()
model = EDSR()

Building EDSR...
Building Complete!


In [13]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=10)
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt and ckpt.model_checkpoint_path:
    loader = tf.train.Saver(var_list=tf.global_variables())
    load_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
    load(loader, sess, ckpt.model_checkpoint_path)
else:
    print('no checkpoints find.')

print('[*] Training Start! [*]')
for epoch in range(n_epochs):
    start_time = time.time()
    if epoch !=0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay ** (epoch // decay_every)
        sess.run(tf.assign(model.lr, lr * new_lr_decay))
        log = " ** new learning rate: %f" % (lr * new_lr_decay)
        print(log)
    elif epoch == 0:
        log = " ** init lr: %f, decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
        print(log)     
    n_batches = len(train_images) // batch_size
    random_list = np.random.permutation(len(train_images))
    for iteration in range(n_batches):
        step_time = time.time()
        idx = random_list[iteration*batch_size:(iteration+1)*batch_size]
        y = train_images[idx]
        y = threading_data(y, prepro_img, mode='train')
        x = []
        for image in y:
            x.append(resize(image, (64, 64, 3)))
        x = np.asarray(x)
        eps = np.random.uniform()
        if eps < 0.8:
            sig = np.random.uniform() / 2 + 0.5
            x = ndimage.gaussian_filter(x, sigma=(0, sig, sig, 0), order=0)
        assert x.shape[0] == y.shape[0]
        _, loss = sess.run([model.train_op, model.loss], feed_dict={model.input: x, model.target: y})
        if (iteration + 1) % (n_batches // 10) == 0:
            print('Iteration: [%3d/%3d] time: %4.4fs, loss: %.8f' % (iteration, n_batches, time.time() - step_time, loss))
    if (epoch + 1) % 1 == 0:
        print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
        random_list = np.random.permutation(len(train_images))
        idx = random_list[0: batch_size]
        sample_images_64 = train_images_64[idx]
        sample_outputs = sess.run([model.out], feed_dict={model.input: sample_images_64})
        sample_outputs = np.reshape(sample_outputs, [batch_size, 256, 256, 3])
        save_images(sample_outputs, [4, 4], 'EDSR_samples_with_noise/train_{:02d}.png'.format(epoch))  
    if (epoch != 0) and (epoch % 4) == 0:
        save(saver, sess, checkpoint_dir, epoch + 100)
        print("[*] Save checkpoints SUCCESS!")
print('[*] Training Complete! [*]')

INFO:tensorflow:Restoring parameters from ./EDSR_checkpoint/model.ckpt-98
Restored model parameters from ./EDSR_checkpoint/model.ckpt-98
[*] Training Start! [*]
 ** init lr: 0.000020, decay_every_epoch: 80, lr_decay: 0.500000


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:143: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Iteration: [ 54/553] time: 1.3611s, loss: 0.04477588
Iteration: [109/553] time: 1.3936s, loss: 0.05451610
Iteration: [164/553] time: 1.3486s, loss: 0.03478532
Iteration: [219/553] time: 1.3717s, loss: 0.03347996
Iteration: [274/553] time: 1.3079s, loss: 0.04197994
Iteration: [329/553] time: 1.4249s, loss: 0.03052877
Iteration: [384/553] time: 1.4110s, loss: 0.03479273
Iteration: [439/553] time: 1.3952s, loss: 0.03543075
Iteration: [494/553] time: 1.3968s, loss: 0.03508663
Iteration: [549/553] time: 1.4569s, loss: 0.03447928
 ** Epoch 0 took 767.173972s


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration: [ 54/553] time: 1.3634s, loss: 0.02858874
Iteration: [109/553] time: 1.3753s, loss: 0.02803499
Iteration: [164/553] time: 1.3677s, loss: 0.03346511
Iteration: [219/553] time: 1.4502s, loss: 0.04270903
Iteration: [274/553] time: 1.3652s, loss: 0.04658763
Iteration: [329/553] time: 1.3813s, loss: 0.04184254
Iteration: [384/553] time: 1.3037s, loss: 0.03056753
Iteration: [439/553] time: 1.4358s, loss: 0.03928852
Iteration: [494/553] time: 1.3909s, loss: 0.03802915
Iteration: [549/553] time: 1.4127s, loss: 0.02703553
 ** Epoch 1 took 762.540313s
Iteration: [ 54/553] time: 1.3978s, loss: 0.03733285
Iteration: [109/553] time: 1.3416s, loss: 0.02962982
Iteration: [164/553] time: 1.3681s, loss: 0.03184985
Iteration: [219/553] time: 1.3813s, loss: 0.03305813
Iteration: [274/553] time: 1.4025s, loss: 0.02975033
Iteration: [329/553] time: 1.4086s, loss: 0.03793976
Iteration: [384/553] time: 1.4266s, loss: 0.03111466
Iteration: [439/553] time: 1.3750s, loss: 0.03985460
Iteration: [494/5

...

Iteration: [384/553] time: 1.3582s, loss: 0.03063460
Iteration: [439/553] time: 1.4018s, loss: 0.03543688
Iteration: [494/553] time: 1.4041s, loss: 0.03119796
Iteration: [549/553] time: 1.4181s, loss: 0.02738809
 ** Epoch 29 took 754.445283s
Iteration: [ 54/553] time: 1.3230s, loss: 0.03418803
Iteration: [109/553] time: 1.4156s, loss: 0.03157027
Iteration: [164/553] time: 1.3747s, loss: 0.03338476
Iteration: [219/553] time: 1.4239s, loss: 0.03386866
Iteration: [274/553] time: 1.3513s, loss: 0.03208373
Iteration: [329/553] time: 1.3650s, loss: 0.02900342
Iteration: [384/553] time: 1.3779s, loss: 0.03572103
Iteration: [439/553] time: 1.3534s, loss: 0.04592399
Iteration: [494/553] time: 1.3356s, loss: 0.03346867
Iteration: [549/553] time: 1.3478s, loss: 0.03255861
 ** Epoch 30 took 755.858593s
Iteration: [ 54/553] time: 1.3638s, loss: 0.03466767
Iteration: [109/553] time: 1.3391s, loss: 0.03280951
Iteration: [164/553] time: 1.3481s, loss: 0.03182948
Iteration: [219/553] time: 1.4031s, los

 ** Epoch 43 took 755.549944s
Iteration: [ 54/553] time: 1.3237s, loss: 0.02839051
Iteration: [109/553] time: 1.3715s, loss: 0.04343010
Iteration: [164/553] time: 1.4371s, loss: 0.03044041
Iteration: [219/553] time: 1.3695s, loss: 0.03104133
Iteration: [274/553] time: 1.3963s, loss: 0.02884216
Iteration: [329/553] time: 1.3758s, loss: 0.02465001
Iteration: [384/553] time: 1.3691s, loss: 0.03128133
Iteration: [439/553] time: 1.4145s, loss: 0.03194743
Iteration: [494/553] time: 1.3733s, loss: 0.02884845
Iteration: [549/553] time: 1.3668s, loss: 0.02713175
 ** Epoch 44 took 761.647827s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3281s, loss: 0.03105503
Iteration: [109/553] time: 1.4286s, loss: 0.02769317
Iteration: [164/553] time: 1.4165s, loss: 0.02679566
Iteration: [219/553] time: 1.3430s, loss: 0.02961426
Iteration: [274/553] time: 1.3688s, loss: 0.03444013
Iteration: [329/553] time: 1.3974s, loss: 0.02656310
Iteration: [384/553] time: 1.

Iteration: [164/553] time: 1.3300s, loss: 0.03586284
Iteration: [219/553] time: 1.4119s, loss: 0.02694698
Iteration: [274/553] time: 1.3574s, loss: 0.03010203
Iteration: [329/553] time: 1.3555s, loss: 0.02343005
Iteration: [384/553] time: 1.3908s, loss: 0.02784032
Iteration: [439/553] time: 1.3709s, loss: 0.04025697
Iteration: [494/553] time: 1.3303s, loss: 0.03673612
Iteration: [549/553] time: 1.4080s, loss: 0.03559919
 ** Epoch 58 took 758.081106s
Iteration: [ 54/553] time: 1.3852s, loss: 0.02602217
Iteration: [109/553] time: 1.4043s, loss: 0.03181669
Iteration: [164/553] time: 1.3333s, loss: 0.02609368
Iteration: [219/553] time: 1.3731s, loss: 0.04064329
Iteration: [274/553] time: 1.3897s, loss: 0.03181833
Iteration: [329/553] time: 1.3885s, loss: 0.02406916
Iteration: [384/553] time: 1.3853s, loss: 0.02881348
Iteration: [439/553] time: 1.4182s, loss: 0.02511350
Iteration: [494/553] time: 1.3604s, loss: 0.02623788
Iteration: [549/553] time: 1.3387s, loss: 0.02591166
 ** Epoch 59 too

Iteration: [384/553] time: 1.3667s, loss: 0.02736916
Iteration: [439/553] time: 1.3694s, loss: 0.03148782
Iteration: [494/553] time: 1.3788s, loss: 0.02925111
Iteration: [549/553] time: 1.3135s, loss: 0.03350364
 ** Epoch 72 took 757.866802s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3587s, loss: 0.02683135
Iteration: [109/553] time: 1.3315s, loss: 0.02628198
Iteration: [164/553] time: 1.3544s, loss: 0.02942470
Iteration: [219/553] time: 1.3830s, loss: 0.02634530
Iteration: [274/553] time: 1.3724s, loss: 0.03474700
Iteration: [329/553] time: 1.3453s, loss: 0.03267606
Iteration: [384/553] time: 1.4187s, loss: 0.02594877
Iteration: [439/553] time: 1.3728s, loss: 0.03058691
Iteration: [494/553] time: 1.3816s, loss: 0.03229861
Iteration: [549/553] time: 1.3774s, loss: 0.03185659
 ** Epoch 73 took 757.392501s
Iteration: [ 54/553] time: 1.3631s, loss: 0.04449625
Iteration: [109/553] time: 1.3731s, loss: 0.03373536
Iteration: [164/553] time: 1.

Iteration: [494/553] time: 1.3358s, loss: 0.03091889
Iteration: [549/553] time: 1.3972s, loss: 0.03537684
 ** Epoch 86 took 759.867572s
Iteration: [ 54/553] time: 1.3780s, loss: 0.03093107
Iteration: [109/553] time: 1.4207s, loss: 0.03379412
Iteration: [164/553] time: 1.4002s, loss: 0.03617986
Iteration: [219/553] time: 1.3763s, loss: 0.02892886
Iteration: [274/553] time: 1.4101s, loss: 0.02341870
Iteration: [329/553] time: 1.3944s, loss: 0.02789592
Iteration: [384/553] time: 1.2947s, loss: 0.03128555
Iteration: [439/553] time: 1.4267s, loss: 0.02414480
Iteration: [494/553] time: 1.3658s, loss: 0.02647141
Iteration: [549/553] time: 1.3830s, loss: 0.02997049
 ** Epoch 87 took 758.457389s
Iteration: [ 54/553] time: 1.3656s, loss: 0.02799056
Iteration: [109/553] time: 1.3915s, loss: 0.03262084
Iteration: [164/553] time: 1.4552s, loss: 0.02741137
Iteration: [219/553] time: 1.4047s, loss: 0.02629965
Iteration: [274/553] time: 1.3904s, loss: 0.03821611
Iteration: [329/553] time: 1.4202s, los

In [14]:
""" Re-Training Parameters """
n_epochs = 20
batch_size = 16
lr = 1e-5
decay_every = 10
lr_decay = 0.5
train_samples_dir = 'EDSR_samples_with_noise'
if os.path.exists(train_samples_dir) == False:
    os.makedirs(train_samples_dir)
checkpoint_dir = './EDSR_checkpoint'

In [15]:
print('[*] Training Start! [*]')
for epoch in range(n_epochs):
    start_time = time.time()
    if epoch !=0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay ** (epoch // decay_every)
        sess.run(tf.assign(model.lr, lr * new_lr_decay))
        log = " ** new learning rate: %f" % (lr * new_lr_decay)
        print(log)
    elif epoch == 0:
        log = " ** init lr: %f, decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
        print(log)     
    n_batches = len(train_images) // batch_size
    random_list = np.random.permutation(len(train_images))
    for iteration in range(n_batches):
        step_time = time.time()
        idx = random_list[iteration*batch_size:(iteration+1)*batch_size]
        y = train_images[idx]
        y = threading_data(y, prepro_img, mode='train')
        x = []
        for image in y:
            x.append(resize(image, (64, 64, 3)))
        x = np.asarray(x)
        eps = np.random.uniform()
        if eps < 1.0:
            sig = np.random.uniform() / 2 + 1.0
            x = ndimage.gaussian_filter(x, sigma=(0, sig, sig, 0), order=0)
        assert x.shape[0] == y.shape[0]
        _, loss = sess.run([model.train_op, model.loss], feed_dict={model.input: x, model.target: y})
        if (iteration + 1) % (n_batches // 10) == 0:
            print('Iteration: [%3d/%3d] time: %4.4fs, loss: %.8f' % (iteration, n_batches, time.time() - step_time, loss))
    if (epoch + 1) % 1 == 0:
        print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
        random_list = np.random.permutation(len(train_images))
        idx = random_list[0: batch_size]
        sample_images_64 = train_images_64[idx]
        sample_outputs = sess.run([model.out], feed_dict={model.input: sample_images_64})
        sample_outputs = np.reshape(sample_outputs, [batch_size, 256, 256, 3])
        save_images(sample_outputs, [4, 4], 'EDSR_samples_with_noise/train_{:02d}.png'.format(epoch + 100))  
    if (epoch != 0) and (epoch % 4) == 0:
        save(saver, sess, checkpoint_dir, epoch + 200)
        print("[*] Save checkpoints SUCCESS!")
print('[*] Training Complete! [*]')

[*] Training Start! [*]
 ** init lr: 0.000010, decay_every_epoch: 10, lr_decay: 0.500000


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:143: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Iteration: [ 54/553] time: 1.3259s, loss: 0.03223538
Iteration: [109/553] time: 1.3652s, loss: 0.03510970
Iteration: [164/553] time: 1.3654s, loss: 0.03340490
Iteration: [219/553] time: 1.4132s, loss: 0.04481713
Iteration: [274/553] time: 1.3648s, loss: 0.03656874
Iteration: [329/553] time: 1.2874s, loss: 0.04993982
Iteration: [384/553] time: 1.3722s, loss: 0.02369100
Iteration: [439/553] time: 1.4153s, loss: 0.04140322
Iteration: [494/553] time: 1.4338s, loss: 0.03832370
Iteration: [549/553] time: 1.3670s, loss: 0.04261090
 ** Epoch 0 took 765.194869s


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration: [ 54/553] time: 1.3700s, loss: 0.03892256
Iteration: [109/553] time: 1.3860s, loss: 0.03617276
Iteration: [164/553] time: 1.3944s, loss: 0.03941011
Iteration: [219/553] time: 1.4202s, loss: 0.02988461
Iteration: [274/553] time: 1.4132s, loss: 0.02707804
Iteration: [329/553] time: 1.4129s, loss: 0.02860604
Iteration: [384/553] time: 1.3442s, loss: 0.03492570
Iteration: [439/553] time: 1.4052s, loss: 0.04109117
Iteration: [494/553] time: 1.3694s, loss: 0.03639216
Iteration: [549/553] time: 1.3953s, loss: 0.03058383
 ** Epoch 1 took 765.520152s
Iteration: [ 54/553] time: 1.3959s, loss: 0.03777520
Iteration: [109/553] time: 1.4149s, loss: 0.03948411
Iteration: [164/553] time: 1.3848s, loss: 0.02825040
Iteration: [219/553] time: 1.4018s, loss: 0.04332276
Iteration: [274/553] time: 1.3524s, loss: 0.03797686
Iteration: [329/553] time: 1.3734s, loss: 0.03512761
Iteration: [384/553] time: 1.3932s, loss: 0.04218156
Iteration: [439/553] time: 1.3364s, loss: 0.04472771
Iteration: [494/5

Iteration: [219/553] time: 1.4035s, loss: 0.03587485
Iteration: [274/553] time: 1.3377s, loss: 0.02760775
Iteration: [329/553] time: 1.3148s, loss: 0.03497642
Iteration: [384/553] time: 1.4016s, loss: 0.03090390
Iteration: [439/553] time: 1.3786s, loss: 0.03396435
Iteration: [494/553] time: 1.4158s, loss: 0.04054211
Iteration: [549/553] time: 1.3833s, loss: 0.02697640
 ** Epoch 15 took 765.873880s
Iteration: [ 54/553] time: 1.3779s, loss: 0.03256973
Iteration: [109/553] time: 1.3435s, loss: 0.03617609
Iteration: [164/553] time: 1.4105s, loss: 0.03280073
Iteration: [219/553] time: 1.3732s, loss: 0.03266993
Iteration: [274/553] time: 1.3661s, loss: 0.02753038
Iteration: [329/553] time: 1.4248s, loss: 0.02630013
Iteration: [384/553] time: 1.3638s, loss: 0.03467460
Iteration: [439/553] time: 1.4962s, loss: 0.03265600
Iteration: [494/553] time: 1.3807s, loss: 0.03327285
Iteration: [549/553] time: 1.2733s, loss: 0.03685452
 ** Epoch 16 took 766.025072s
The checkpoint has been created.
[*] Sa

In [19]:
x = train_images_64[0:16]
x = ndimage.gaussian_filter(x, sigma=(0, 1.0, 1.0, 0), order=0)
sample_outputs = sess.run([model.out], feed_dict={model.input: x})
sample_outputs = np.reshape(sample_outputs, [batch_size, 256, 256, 3])
save_images(sample_outputs, [4, 4], 'EDSR_samples_with_noise/Inference.png') 

/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
